In [1]:
import os
os.chdir('../src')

In [5]:
import logging, datetime
from pathlib import Path
import torch
import pandas as pd

from datasets import create_train_loader, create_valid_loader
from datasets.train_dataset import trainMammo
from datasets.transforms import get_train_transform
from configs import cfg
from modeling.model import kaggleBCModel
from engine.fitter import Fitter

In [4]:
from modeling.next_vit import nextvit_base
model = nextvit_base()

initialize_weights...


In [8]:
checkpoint = torch.load('/home/giorgio/Scrivania/kaggle/rnsa-mammo-kaggle/data/nextvit_base_in1k_384.pth')
checkpoint.keys()

dict_keys(['model', 'optimizer', 'lr_scheduler', 'epoch', 'args'])

In [12]:
model.load_state_dict(checkpoint['model'])

<All keys matched successfully>

In [15]:
model.proj_head = torch.nn.Linear(in_features=1024, out_features=1)
model

NextViT(
  (stem): Sequential(
    (0): ConvBNReLU(
      (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): ReLU(inplace=True)
    )
    (1): ConvBNReLU(
      (conv): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (norm): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): ReLU(inplace=True)
    )
    (2): ConvBNReLU(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): ReLU(inplace=True)
    )
    (3): ConvBNReLU(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (norm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): ReLU(inplace=Tr

In [21]:
train_df = pd.read_csv(
    '../data/train_folds.csv'
)

train_loader = create_train_loader(
    cfg,
    df=train_df[train_df['fold'] != cfg.INPUT.VALID_FOLD].sample(100)
)

valid_loader = create_valid_loader(
    cfg,
    df=train_df[train_df['fold'] == cfg.INPUT.VALID_FOLD].sample(100)
)

#model = kaggleBCModel(cfg)

In [22]:
trnsf = get_train_transform(cfg)
train_df = trainMammo(
    cfg, 
    df=train_df[train_df['fold'] != cfg.INPUT.VALID_FOLD].sample(100),
    transforms = trnsf
)

In [23]:
img, label, _, _ = train_df[0]

In [24]:
img.unsqueeze(0)

tensor([[[[ 0.4588,  0.4824,  0.4902,  ..., -1.0000, -1.0000, -1.0000],
          [ 0.6157,  0.5529,  0.4824,  ..., -1.0000, -1.0000, -1.0000],
          [ 0.4980,  0.5373,  0.5373,  ..., -1.0000, -1.0000, -1.0000],
          ...,
          [ 0.6078,  0.4824,  0.4902,  ..., -1.0000, -1.0000, -1.0000],
          [ 0.1373, -0.1529, -0.4667,  ..., -1.0000, -1.0000, -1.0000],
          [-0.9137, -0.9686, -0.9922,  ..., -1.0000, -1.0000, -1.0000]],

         [[ 0.4588,  0.4824,  0.4902,  ..., -1.0000, -1.0000, -1.0000],
          [ 0.6157,  0.5529,  0.4824,  ..., -1.0000, -1.0000, -1.0000],
          [ 0.4980,  0.5373,  0.5373,  ..., -1.0000, -1.0000, -1.0000],
          ...,
          [ 0.6078,  0.4824,  0.4902,  ..., -1.0000, -1.0000, -1.0000],
          [ 0.1373, -0.1529, -0.4667,  ..., -1.0000, -1.0000, -1.0000],
          [-0.9137, -0.9686, -0.9922,  ..., -1.0000, -1.0000, -1.0000]],

         [[ 0.4588,  0.4824,  0.4902,  ..., -1.0000, -1.0000, -1.0000],
          [ 0.6157,  0.5529,  

In [27]:
import torch
torch.sigmoid(model(img.unsqueeze(0).float()))

tensor([[0.5019, 0.5016]], grad_fn=<SigmoidBackward0>)

In [4]:
Path(cfg.OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
Path(cfg.OUTPUT_DIR).mkdir(parents=True, exist_ok=True)
Path(cfg.OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

#Istanzio il logger
path_logger = os.path.join(
    cfg.OUTPUT_DIR, f'train-{datetime.datetime.now()}.log'
)

logging.basicConfig(filename=path_logger, level=logging.DEBUG)
logger = logging.getLogger()

In [5]:
fit_engine = Fitter(
    model,
    cfg,
    train_loader,
    valid_loader,
    logger
)

In [6]:
fit_engine.load(
    '../experiments/resnet34_2022-12-30/weights/resnet34_best.pth'
)

In [7]:
fit_engine.best_final_score

0.13333333333333336